In [41]:
import pandas as pd 
import numpy as np 
import torch 
import torch.nn.functional as fn 
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [12]:
df = pd.read_csv('data_banknote_authentication.txt', sep=',', header=None)

In [14]:
df.describe()

,0,1,2,3,4
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [15]:
X_features = df[[0,1,2,3]].values 
y_labels = df[4].values

In [16]:
np.bincount(y_labels)

array([762, 610])

In [18]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.labels = torch.tensor(y, dtype=torch.float32)
        self.features = torch.tensor(X, dtype=torch.float32)
    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return x, y 
    def __len__(self):
        return self.labels.shape[0]

In [29]:
train_size = int(X_features.shape[0]*0.7)
valid_size = int(X_features.shape[0]*0.2)
test_size =  X_features.shape[0] - (train_size+valid_size)


In [40]:
dataset = MyDataset(X_features, y_labels)
torch.manual_seed(123)
train_set, val_set, test_set = torch.utils.data.random_split(
    dataset, 
    lengths=[train_size, valid_size, test_size]
)
train_loader = DataLoader(
    train_set, 
    shuffle=True,
    batch_size=10
)
val_loader = DataLoader(
    val_set, 
    shuffle=True, 
    batch_size=10
    
)
test_loader = DataLoader(
    test_set, 
    shuffle=True, 
    batch_size = 10
)

In [37]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()
        # initialize weight and bias terms
        self.linear = torch.nn.Linear(num_features, 1)
    def forward (self, x):
        logits = self.linear(x)
        probas = torch.sigmoid(logits)
        return probas

In [51]:
len([i for i in range(50, 200, 20)])

8

In [53]:
models = {}
for num_epochs in range(10, 200, 10):
    for lr in np.linspace(0.001, 0.01, 10):
        torch.manual_seed(123)
        model = LogisticRegression(num_features=4)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        for epoch in range(num_epochs):
            model = model.train()
            for batch_idx, (features, labels) in enumerate(train_loader):
                probas = model.forward(features)
                loss = fn.binary_cross_entropy(probas, labels.view_as(probas))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
#                 if not batch_idx % 20: # log every 20th batch
#                     print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
#                            f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
#                            f' | Loss: {loss:.2f}')
        models[f"{lr}-{num_epochs}"] = {"model":model, "loss":loss}

In [78]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    
    correct = 0.0
    total_examples = 0
    
    for idx, (features, class_labels) in enumerate(dataloader):
        
        with torch.inference_mode():
            probas = model(features)
        
        pred = torch.where(probas > 0.5, 1, 0)
        lab = class_labels.view(pred.shape).to(pred.dtype)

        compare = lab == pred
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

In [79]:
best_model = None
accuracy = 0
for name, model in models.items():
    accuracy_new = compute_accuracy(model["model"], val_loader)
    if accuracy_new>accuracy:
        best_model=name
        accuracy = accuracy_new

In [80]:
best_model, accuracy

('0.01-60', tensor(0.9964))

In [81]:
best_model=models[best_model]["model"]

In [82]:
compute_accuracy(best_model, val_loader)

tensor(0.9964)

In [83]:
compute_accuracy(best_model, train_loader)

tensor(0.9854)

In [84]:
compute_accuracy(best_model, test_loader)

tensor(0.9855)